In [1]:
from cv2 import cv2 as cv
import numpy as np
from local_tools import *
import torchvision
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import transforms

%load_ext autoreload
%autoreload 2

In [2]:
frames, totalFrames = read_frames('../videos/traffic2.mp4')

In [3]:
device = torch.device("cuda:0") #Подрубайтунг

In [4]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True).to(device)

In [5]:
convert_tensor = transforms.ToTensor()

In [7]:
model = model.eval()

boxes = []
threshold = 0.5

playing = False

frame_number = 0


def set_frame_number(x):
    global frame_number, totalFrames
    frame_number = x % totalFrames


cv.namedWindow('control')
cv.createTrackbar('frame', 'control', 1, int(totalFrames) - 1, set_frame_number)

with torch.no_grad():
    model = model.cuda()
    while True:
        
        if playing:
            frame_number = (frame_number + 1) % totalFrames

        frame = frames[frame_number].copy()

        tenzor = torch.tensor(np.swapaxes(np.swapaxes(frame, 0, 2), 1, 2)).float() / 255.
        pred = model([tenzor.to(device)])[0]
        
        if (pred!=None):
            for i in range(len(pred["labels"])):
                if pred["scores"][i] >= threshold and pred["labels"][i] == 1:
                    boxes = pred['boxes'][i]
                    frame = cv.rectangle(frame, (int(boxes[0]), int(boxes[1]) ), ( int(boxes[2]),int(boxes[3])), (255,0,100), 2)

        cv.imshow('img', frame)

        k = cv.waitKey(1) & 0xFF

        if k == ord('e'):
            playing = not playing

        if k == 27:
            cv.destroyAllWindows()
            break

In [7]:
cv.destroyAllWindows()